In [2]:
import os

class Country():
    """Country class which specifies the (top 10) ranking of a country.
    Optionally gives the score this country achieved in the semifinal."""
    def __init__(self, country, ranking_list, semifinal_score=0):
        self.country = country
        self.ranking_list = ranking_list
        self.semifinal_score = semifinal_score

    def get_ranking_list(self):
        return self.ranking_list

    def get_semifinal_score(self):
        return self.semifinal_score

    def get_ith_ranking(self, i):
        return self.ranking_list[i-1]

class Rule():
    """Rule class consisting of a scoring vector and a vector length.
    Scoring vector is specified from largest to smallest score.
    Will be expanded once the thesis gets to more advanced
    material."""
    def __init__(self, scoring_vector, vector_length):
        self.scoring_vector = scoring_vector
        self.vector_length = vector_length

    def get_scoring_vector(self):
        return self.scoring_vector

    def get_vector_length(self):
        return self.vector_length

class Contest():
    """Contest class includes everything to compute the original outcome
    of a certain year's ESC. Later will also be able to solve the problem(s)
    specified in the thesis proposal."""
    def __init__(self, country_list, voting_rule, nr_of_semifinals=0):
        self.country_list = country_list
        self.voting_rule = voting_rule
        self.nr_of_countries = len(country_list)
        self.nr_of_semifinals = nr_of_semifinals
        
    def compute_result(self):
        """Computes the result and outputs a dict with each country that scored
        more than 0 points."""
        result = dict()
        rule_vector = self.voting_rule.get_scoring_vector()
        rule_length = self.voting_rule.get_vector_length()
        for country in self.country_list:
            ranking_list = country.get_ranking_list()
            rank = 0
            for c in ranking_list:
                if rank + 1 > rule_length:
                    break
                if c not in result:
                    result[c] = 0
                result[c] += rule_vector[rank]
                rank += 1

        return result

    def print_result(self):
        """Prints the result in a ranking list format."""
        result = self.compute_result()
        print("### ESC " + 'whatever' + " ###")
        for element in sorted(result, key=result.get, reverse=True):
            print(element, result[element])
        print()


In [3]:
import csv

def import_scores(range1, range2, file_name = 'votes.csv'):
    """ Reads and puts the votes for the finals
    from range1 to range2 in a dictionary. This dict has 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)"""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC = dict()
    for year in range(range1, range2):
        ESC[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(range1, range2) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC[row[0]]:
            ESC[row[0]][row[2]] = dict()
        ESC[row[0]][row[2]][row[3]] = int(row[6])
        
    scores_to_rankings(ESC)
    return ESC

def scores_to_rankings(ESC_dict):
    """ ESC dict contains a year dict, which contains country dicts with a dict
    using scored_country/score tuples. This function turns that into a list
    ranking the countries instead. """
    for year in ESC_dict:
        for country in ESC_dict[year]:
            score_dict = ESC_dict[year][country]
            ranking_list = []
            for scored_country in sorted(score_dict, key=score_dict.get, reverse=True):
                ranking_list.append(scored_country)
            ESC_dict[year][country] = ranking_list
    pass

def dict_to_classes(d, voting_rule):
    """Takes as input a dict outputted by import_scores() and a scoring rule,
    and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(1975,2016):
        ESC_classified[str(year)] = None
        
    for year in d:
        country_list = []
        for country in d[year]:
            new_country = Country(country, d[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)
    
    return ESC_classified

In [4]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)

ESC_classified = dict_to_classes(import_scores(1975, 2016), rule_1975_2015)
ESC_classified['1975'].print_result()

### ESC whatever ###
nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3



In [5]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)

ESC_old = dict_to_classes(import_scores(1962, 1967), rule_1962)
ESC_old['1963'].print_result()
ESC_old['1964'].print_result()
ESC_old['1965'].print_result()
ESC_old['1966'].print_result()

### ESC whatever ###
ch 20
dk 20
it 16
gb 11
mc 9
fr 9
at 5
lu 3
be 2
de 1

### ESC whatever ###
it 30
gb 12
fr 10
mc 10
lu 9
at 7
fi 6
no 4
dk 3
nl 2
be 2
es 1

### ESC whatever ###
lu 21
gb 16
fr 15
it 11
at 10
ie 7
dk 6
ch 5
mc 5
se 4
nl 3
yu 2
no 1
pt 1

### ESC whatever ###
at 20
no 10
se 10
ie 9
be 9
ch 8
yu 6
es 6
lu 5
de 5
fi 5
gb 5
pt 4
dk 3
nl 2
fr 1



In [17]:
def import_scores_2015_and_beyond(file_name = 'votes.csv'):
    """Gives two rankings and one point total."""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC_jury, ESC_tele, ESC_total = dict(), dict(), dict()

    for year in range(2016,2020):
        ESC_jury[str(year)], ESC_tele[str(year)] = dict(), dict()
        ESC_total[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(2016,2020) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC_jury[row[0]] and row[8] != 0:
            ESC_jury[row[0]][row[2]] = dict()
        if row[2] not in ESC_tele[row[0]] and row[7] != 0:
            ESC_tele[row[0]][row[2]] = dict()
        if row[2] not in ESC_total[row[0]]:
            ESC_total[row[0]][row[2]] = dict()
        if row[8] != 0:
            ESC_jury[row[0]][row[2]][row[3]] = int(row[8])
        if row[7] != 0:
            ESC_tele[row[0]][row[2]][row[3]] = int(row[7])
        ESC_total[row[0]][row[2]][row[3]] = int(row[6])

    scores_to_rankings(ESC_jury)
    scores_to_rankings(ESC_tele)

    return ESC_jury, ESC_tele, ESC_total

def pre_2015_ify(jury, tele, total, voting_rule=Rule([12,10,8,7,6,5,4,3,2,1], 10)):
    result = dict()
    rule_vector = voting_rule.get_scoring_vector()
    rule_length = voting_rule.get_vector_length()
    
    # https://www.geeksforgeeks.org/python-program-to-swap-two-elements-in-a-list/
    def swap_positions(l, elem1, elem2):
        pos1, pos2 = l.index(elem1), l.index(elem2)
        l[pos1], l[pos2] = l[pos2], l[pos1] 
        return l
    
    def bubble_sort_tuple_list(l):
        for i in range(len(l) - 1):
            if l[i][1] == l[i + 1][1]:
                if tele.index(l[i+1][0]) > tele.index(l[i][0]):
                    swap_positions(tuple_list, tuple_list[i+1], tuple_list[i])
    
    pre_2015_ranking = []
    tuple_list = []
    
    for element in sorted(total, key=total.get, reverse=True):
        tuple_list.append((element, total[element]))
        
    for i in range(5):
        bubble_sort_tuple_list(tuple_list)
    
    ranking_list = []
    for i in range(len(tuple_list)):
        ranking_list.append(tuple_list[i][0])
        
    print(total)
    print(ranking_list)
    return ranking_list
        
def dicts_to_2015_classes(d_jury, d_tele, d_total, voting_rule):
    """Takes as input two dicts outputted by import_scores_2015_and_beyond() and
    a scoring rule, and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(2016,2020):
        ESC_classified[str(year)] = None

    for year in d_jury:
        country_list = []
        for country in d_jury[year]:
            new_country = Country2015(country,
                                      d_jury[year][country], 
                                      d_tele[year][country],
                                      d_total[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)

    return ESC_classified

class Country2015(Country):
    def __init__(self, country, jury, tele, total, semifinal_score=0):
        self.country = country
        self.ranking_list_jury = jury
        self.ranking_list_tele = tele
        self.ranking_list = pre_2015_ify(jury, tele, total)
        self.semifinal_score = semifinal_score

In [18]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
d_jury, d_tele, d_total = import_scores_2015_and_beyond()
ESC_classified_2015 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)

{'ua': 6, 'au': 24, 'ru': 14, 'bg': 12, 'se': 3, 'fr': 10, 'am': 2, 'pl': 5, 'lt': 4, 'mt': 2, 'il': 3, 'it': 18, 'hu': 2, 'es': 6, 'gb': 5}
['au', 'it', 'ru', 'bg', 'fr', 'es', 'ua', 'gb', 'pl', 'lt', 'il', 'se', 'mt', 'hu', 'am']
{'ua': 10, 'au': 5, 'ru': 14, 'bg': 7, 'se': 2, 'fr': 19, 'pl': 1, 'be': 4, 'mt': 13, 'at': 4, 'hu': 3, 'ge': 18, 'cy': 12, 'es': 3, 'cz': 1}
['fr', 'ge', 'ru', 'mt', 'cy', 'ua', 'bg', 'au', 'be', 'at', 'es', 'hu', 'se', 'cz', 'pl']
{'ua': 10, 'ru': 5, 'bg': 18, 'fr': 13, 'lt': 8, 'be': 24, 'nl': 3, 'mt': 6, 'at': 3, 'il': 10, 'es': 7, 'hr': 1, 'gb': 8}
['be', 'bg', 'fr', 'il', 'ua', 'lt', 'gb', 'es', 'mt', 'ru', 'nl', 'at', 'hr']
{'ua': 10, 'au': 15, 'ru': 11, 'bg': 5, 'se': 15, 'fr': 8, 'am': 2, 'pl': 12, 'lt': 1, 'be': 5, 'nl': 6, 'mt': 10, 'rs': 4, 'hr': 6, 'cz': 4, 'de': 2}
['au', 'se', 'pl', 'ru', 'mt', 'ua', 'fr', 'hr', 'nl', 'be', 'bg', 'cz', 'rs', 'am', 'de', 'lt']
{'ua': 20, 'au': 9, 'ru': 24, 'bg': 16, 'fr': 4, 'pl': 5, 'lt': 5, 'mt': 11, 'il': 9,

In [16]:
print(ESC_classified_2015)

{'2016': <__main__.Contest object at 0x7ff1cecaec18>, '2017': <__main__.Contest object at 0x7ff1ced175c0>, '2018': <__main__.Contest object at 0x7ff1ced17f60>, '2019': <__main__.Contest object at 0x7ff1ced228d0>}
